In [ ]:

import sqlite3 as sq

def sql_start():
    global base, cur
    base = sq.connect(r'C:\Users\Roman Andreevich\pythonProject21\post.db')
    cur = base.cursor()
    if base:
        print('DBase connected!')
    base.execute('CREATE TABLE IF NOT EXISTS users_main(post_id PRIMARY KEY,user_id INT, date_of_post VARCHAR, likes INT, comments INT, reposts INT, text VARCHAR )')
    base.execute('CREATE TABLE IF NOT EXISTS priznavashki(post_id PRIMARY KEY,user_id INT, date_of_post VARCHAR, likes INT, comments INT,reposts INT,text VARCHAR )')
    base.execute('CREATE TABLE IF NOT EXISTS likes_comments(user_id PRIMARY KEY, likes INT, comments INT )')
    base.execute('CREATE TABLE IF NOT EXISTS all_comments(comment_id PRIMARY KEY,user_id INT,post_id INT, likes INT,text VARCHAR, date_of_comment VARCHAR,likes_id INT )')
    base.commit()

def add_post(id,owner_id,date,likes,comments,reposts,text):
    cur.execute('INSERT INTO priznavashki VALUES (?,?,?,?,?,?,?)', (id,owner_id,date,likes,comments,reposts,text))
    base.commit()

def add_post_search(id,owner_id,date,likes,comments,reposts,text):
    cur.execute('INSERT INTO users_main VALUES (?,?,?,?,?,?,?)', (id,owner_id,date,likes,comments,reposts,text))
    base.commit()

def add_user_likes(id,likes,comments):
    cur.execute('INSERT INTO likes_comments VALUES (?,?,?)', (id,likes,comments))
    base.commit()

def check(data):
    dat = cur.execute('SELECT likes,comments FROM likes_comments WHERE user_id== ?',(data,)).fetchone()
    return dat

def add_something_blyat_uzhe_i_zaebalsya(user_id):
    cur.execute('UPDATE likes_comments SET likes==imp.likes+1 FROM( SELECT likes  FROM likes_comments WHERE user_id== ?) AS imp WHERE user_id== ?',(user_id,user_id))
    base.commit()

def add_comment(id,user_id,post_id,likes,text,data,lk_array):
    cur.execute('INSERT INTO all_comments VALUES (?,?,?,?,?,?,?)', (id,user_id,post_id,likes,text,data,lk_array))
    base.commit()

sql_start()

In [ ]:
import vk_api
from datetime import datetime
def auth_handler():
    key = input("Enter authentication code: ")
    remember_device = True
    return key, remember_device

def get_public_posts(pub_id):
    login, password = '+79539152617', 'Ukulele12!'
    vk_session = vk_api.VkApi(login, password,auth_handler=auth_handler)

    try:
        vk_session.auth(token_only=True)
    except vk_api.AuthError as error_msg:
        print(error_msg)
        return

    tools = vk_api.VkTools(vk_session)
    count =1
    wall = tools.get_all_iter('wall.get', 50, {'owner_id':pub_id})
    for i in range(1000000):
        post = next(wall)
        try:
            if post['signer_id']!=post['owner_id']:
                add_post(post['id'],post['signer_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'],post['reposts']['count'],post['text'])
        except:
            add_post(post['id'],post['owner_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'],post['reposts']['count'],post['text'])
get_public_posts(-59518047)

In [ ]:
from time import sleep
import requests
from datetime import datetime
def get_data(method, params_method, offset=0, count=200,lenm=200):
    params = {
        'access_token': 'vk1.a.rjAJ3MCcjB6HI_Lk9Ala-MRkgeCaMDWJDFN-7VpyT0fWGBiRcD1pv_sYziEofBnVf-O_Ji8CLTWS2QaY_vNo4R2yFXWGKgUP8ylCL9OFtdIL8teqI27T5QM8uYqfR5tsudgamNqqwEZ3IWiDouziuM0w3tc8iI5qqBPLU9WTVvew-9WpI5OWmb9VmevQiRg6dn3n06hMnaseC6Lkf3iIgQ',
        'v': 5.131,
        'count': count,
        'offset': offset,
    }
    for key, item in params_method.items():
        params[key] = item
    data=[]
    response = requests.get(f'https://api.vk.com/method/{method}', params=params)
    try:
        response_data = response.json()['response']['items']
        data.extend(response_data)
    except KeyError:
        pass
        params['offset']+=params['count']
    if method == 'wall.get':
        print(f'Log: Get info from owner_id: {params["owner_id"]}. He has {len(data)} posts on a wall')
    return data

In [ ]:
method = 'newsfeed.search'
dic = ['спбгу','spbu']
params_search={
    'q':'spbu'
}

data = get_data(method,params_search)

for i,post in enumerate(data):
    try:
        add_post_search(post['id'],post['owner_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'],post['reposts']['count'],post['text'])
    except:
        pass


In [ ]:
import pandas as pd
df = pd.read_sql_query("SELECT * from users_main", base)

In [ ]:

def get_likes(df):
    for id in df['post_id']:
        params_get_likes = {
                'type': 'post',
                'owner_id': -59518047 ,
                'item_id': id,
                'friends_only': 0
            }
        method = 'likes.getList'
        data = get_data(method,params_get_likes)
        for i in data:
            try:
                add_user_likes(i,1,0)
            except:
                sqll = 'UPDATE likes_comments SET likes==imp.likes+1 FROM( SELECT likes  FROM likes_comments WHERE user_id== ?) AS imp WHERE user_id== ?'
                cur.execute(sqll,(i,i))
                base.commit()


In [ ]:
def get_comments_stat(df):
    for id in df['post_id']:
        params_get_comments = {
                'preview_length': 0,
                'owner_id': -59518047 ,
                'post_id': id,
                'need_likes': 1,
            }
        method = 'wall.getComments'
        data = get_data(method,params_get_comments)
        for i in data:
            try:
                add_user_likes(i['from_id'],0,1)
            except:
                sqll = 'UPDATE likes_comments SET comments==imp.comments+1 FROM( SELECT comments  FROM likes_comments WHERE user_id== ?) AS imp WHERE user_id== ?'
                cur.execute(sqll,(i['from_id'],i['from_id']))
                base.commit()

In [ ]:
k=0
for id in df['post_id']:
    k+=1
    params_get_comments = {
            'preview_length': 0,
            'owner_id': -59518047 ,
            'post_id': 409058,
            'need_likes': 1,
            'thread_items_count':10
        }
    method = 'wall.getComments'
    data = get_data(method,params_get_comments)
    for i in data:
        lk_array=[]
        try:
            for j in i['thread']['items']:
                lk_array=[]
                try:
                    params_get_likes = {
                    'type': 'comment',
                    'owner_id': -59518047 ,
                    'item_id': j['id'],
                    'friends_only': 0
                            }
                    method = 'likes.getList'
                    lk_array = get_data(method,params_get_likes)
                    add_comment(j['id'],j['from_id'],j['post_id'],j['likes']['count'],j['text'],datetime.fromtimestamp(j['date']).strftime('%Y-%m-%d'),str(lk_array))
                except:
                    try:
                        add_comment(j['id'],j['from_id'],j['post_id'],j['likes']['count'],j['text'],datetime.fromtimestamp(j['date']).strftime('%Y-%m-%d'),str(lk_array))
                    except:
                        pass
        except:
            pass
        lk_array=[]
        try:
            params_get_likes = {
            'type': 'comment',
            'owner_id': -59518047 ,
            'item_id': i['id'],
            'friends_only': 0,
            'filter':'likes'
                    }
            method = 'likes.getList'
            lk_array = get_data(method,params_get_likes)
            add_comment(i['id'],i['from_id'],i['post_id'],i['likes']['count'],i['text'],datetime.fromtimestamp(i['date']).strftime('%Y-%m-%d'),str(lk_array))
        except:
            try:
                add_comment(i['id'],i['from_id'],i['post_id'],i['likes']['count'],i['text'],datetime.fromtimestamp(i['date']).strftime('%Y-%m-%d'),str(lk_array))
            except:
                pass
    if k%500==0:
        print(k)